In [13]:
import sys
import os
from pathlib import Path

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

GNDTRUTH = Path('../data/raw/COMMON_images_masks/')
RAWIMGS = Path('../data/raw/GROUP_images/')
DATA = (RAWIMGS.parent).parent
MASKS = DATA/'masks'

import SimpleITK as sitk

from utils.visualization import *
from utils.segmentation import *
from utils.crop import *
from utils.registration import *

In [2]:
fix_im = sitk.ReadImage(GNDTRUTH/'common_40_image.nii.gz')
mov_im = sitk.ReadImage(RAWIMGS/'g1_54_image.nii.gz')
fix_msk = sitk.ReadImage(MASKS/'common_40_regmask.nii.gz')
mov_msk = sitk.ReadImage(MASKS/'g1_54_mask.nii.gz')

# Arrays from images
fix_im_data = sitk.GetArrayFromImage(fix_im)
fix_msk_data = sitk.GetArrayFromImage(fix_msk)
mov_im_data = sitk.GetArrayFromImage(mov_im)
mov_msk_data = sitk.GetArrayFromImage(mov_msk)


In [3]:
show_interactive(mov_im_data, 'coronal', mov_msk_data)

interactive(children=(IntSlider(value=256, description='slc', max=511), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_slice(arr, slc, mask=None)>

In [4]:
fix_bonemask = mask_from_hu(fix_im, hu_min=185, hu_max=500, closing_kernel_size=10, verbose=True)
fix_bonemask_data = sitk.GetArrayFromImage(fix_bonemask)

show_interactive(fix_im_data, 'coronal', fix_bonemask_data)

Generating mask from HU thresholding with HU_min=185, HU_max=500...
Mask generated!


interactive(children=(IntSlider(value=256, description='slc', max=511), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_slice(arr, slc, mask=None)>

In [5]:
# Clean the mask so that it does not include parts of the medical table
clean_fix_bonemask = clean_bone_mask(fix_bonemask)
fix_bonemask_data = sitk.GetArrayFromImage(clean_fix_bonemask)

In [6]:
show_interactive(fix_im_data, 'coronal', fix_bonemask_data)

interactive(children=(IntSlider(value=256, description='slc', max=511), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_slice(arr, slc, mask=None)>

In [7]:
# Crop image and mask to region of interest
fix_im_cropped, fix_msk_cropped = crop_roi_from_mask(fix_im, clean_fix_bonemask)
fix_im_cropped = crop_lateral(fix_im_cropped, 'r')
fix_msk_cropped = crop_lateral(fix_msk_cropped, 'r')
fix_im_cr_data = sitk.GetArrayFromImage(fix_im_cropped)
fix_msk_cr_data = sitk.GetArrayFromImage(fix_msk_cropped)
show_interactive(fix_im_cr_data, 'coronal', fix_msk_cr_data)

interactive(children=(IntSlider(value=136, description='slc', max=271), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_slice(arr, slc, mask=None)>

In [8]:
mov_im_cropped, mov_msk_cropped = crop_roi_from_mask_multi(mov_im, mov_msk)
mov_im_cropped_data = sitk.GetArrayFromImage(mov_im_cropped)
mov_msk_cropped_data = sitk.GetArrayFromImage(mov_msk_cropped)
show_interactive(mov_im_cropped_data, 'coronal', mov_msk_cropped_data)

interactive(children=(IntSlider(value=123, description='slc', max=246), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_slice(arr, slc, mask=None)>

In [9]:
lin_tfm1 = est_lin_transf(fix_im_cropped, mov_im_cropped, fix_msk_cropped)

itk::simple::CompositeTransform
 CompositeTransform (0x38a4af60)
   RTTI typeinfo:   itk::CompositeTransform<double, 3u>
   Reference Count: 1
   Modified Time: 630371
   Debug: Off
   Object Name: 
   Observers: 
     none
   TransformQueue: 
   >>>>>>>>>
   AffineTransform (0x39057530)
     RTTI typeinfo:   itk::AffineTransform<double, 3u>
     Reference Count: 1
     Modified Time: 630217
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       0.985825 0.0279998 -0.0528489 
       0.0784387 0.974097 -0.00591919 
       -0.07782 0.100621 0.944333 
     Offset: [19.6883, 12.4586, -1536.83]
     Center: [-79.3516, -127.672, 842.698]
     Translation: [-27.2974, 4.55335, -1590.41]
     Inverse: 
       1.02168 -0.0352509 0.0569566 
       -0.0817057 1.02875 0.0018757 
       0.0928998 -0.11252 1.06344 
     Singular: 0
   TransformsToOptimizeFlags: 
           1 
   TransformsToOptimizeQueue: 
   PreviousTransformsToOptimizeUpdateTime: 0

--------
Optimizer 

In [11]:
mov_im_reg = apply_lin_transf(mov_im, lin_tfm1, fix_im)

In [12]:
mov_im_reg_data = sitk.GetArrayFromImage(mov_im_reg)
show_interactive_overlay(fix_im_data, mov_im_reg_data, 'coronal')

interactive(children=(IntSlider(value=256, description='slc', max=511), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_overlay(fix_arr, mov_arr, slc)>